## 🎥🍿🍬 Movie Recommender System 🍬🍿🎥

#### Group 2: Anish Adimulam, Saad Ahmad, Sydney Johnson, Yvette Soto, Trinity Thao, Saye Xavier

In this final project, we create a recommender system that showcases movies to viewers based on their rating history. The movies shown will be based on the type of rating(s) the user has given per genre of movie they watched. The goal of this final project is to showcase the skills we have learned throughout the semester to create a usable product with our dataset.

Our dataset is the __Movielens Latest Small__ dataset. The dataset can be found at: https://grouplens.org/datasets/movielens/latest/

This dataset consists of four _csv_ files:
- __movies.csv__
- __ratings.csv__
- __tags.csv__
- __links.csv__

_In our project, we will only use the __movies.csv__ and __ratings.csv__ files to build our system._

The __movies.csv__ consists of the following:
- `movieID`
- `title`
- `genres`

The __ratings.csv__ consists of the following:
- `userID`
- `movieID`
- `rating`
- `timestamp`


Our methodology for this project is as follows:
- Perform __Pre-processing__
    - Check for missing values
    - Make sure IDs match between the movies.csv file and the ratings.csv file

- Conduct __Preference Analysis__
    - Identify movies with 4 stars or higher from the dataset and extract their genres
    - Count amount of times each genre shows up in a user's list
    - Create a preference vector based on their rating counts

- Finalize with __Cosine Similarity__
    - Normalize preference vector
    - Create a genre vector representing unwatched movies' genres (one-hot encoding will be used to represent genres)
    - Calculate cosine similarity between user's preference vector and each unwatched movie's genre vector
    - Store similarity scores for each unwatched movie and sort the movies in descending order based on score
    - Select the top K highest-ranked movies as the user's personalized recommendations

In [3]:
#Imports
import pandas as pd

In [4]:
#Load the data files
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

### Pre-Processing

In [5]:
#Check if any data is missing
movies_missing = movies.isnull().sum()
ratings_missing = ratings.isnull().sum()
print("Checking for missing data:")
print("Movies missing data:", movies_missing)
print("Ratings missing data:", ratings_missing)

Checking for missing data:
Movies missing data: movieId    0
title      0
genres     0
dtype: int64
Ratings missing data: userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [6]:
#Checking if movies in ratings exist in movies file
rating_movie_ids = set(ratings['movieId'])
movie_ids = set(movies['movieId'])
movies_only_in_ratings = rating_movie_ids - movie_ids
if len(movies_only_in_ratings) == 0:
    print("All movie IDs match!")
else:
    print("Found", len(movies_only_in_ratings), "movies in ratings that don't exist in movies.csv")

All movie IDs match!


In [7]:
#Convert timestamp to date
from datetime import datetime
ratings['date'] = ratings['timestamp'].apply(lambda x: datetime.fromtimestamp(x))

### Preference Analysis

In [8]:
#Find movies rated 4 stars or higher
good_ratings = ratings[ratings['rating'] >= 4.0]
print("Number of 4+ star ratings:", len(good_ratings))

Number of 4+ star ratings: 48580


In [9]:
#Split genres into lists
movies['genre_list'] = movies['genres'].str.split('|')

In [10]:
#Get all unique genres
all_genres = []
for index, row in movies.iterrows():
    genres = row['genre_list']
    for genre in genres:
        if genre not in all_genres:
            all_genres.append(genre)
all_genres.sort()
print("Found", len(all_genres), "unique genres")

Found 20 unique genres


In [11]:
# Create preference vectors for each user
user_preferences = {}
unique_users = ratings['userId'].unique()

for user in unique_users:
    # Get their highly rated movies
    user_good_ratings = good_ratings[good_ratings['userId'] == user]
    
    # Count each genre
    genre_counts = {}
    for genre in all_genres:
        genre_counts[genre] = 0
    
    # For each highly rated movie, add to genre counts
    for index, rating_row in user_good_ratings.iterrows():
        movie_id = rating_row['movieId']
        movie_row = movies[movies['movieId'] == movie_id]
        
        if len(movie_row) > 0:
            # Instead of .iloc[0], we'll convert to a list and take the first item
            movie_genres = list(movie_row['genre_list'])[0]
            for genre in movie_genres:
                genre_counts[genre] = genre_counts[genre] + 1

    # Creating the vector
    vector = []
    for genre in all_genres:
        vector.append(genre_counts[genre])
    
    user_preferences[user] = vector

### User 1 example (can remove) just put this here to make sure it worked.

In [12]:
# Show user preferences
print("\nExample - User 1 preferences:")
print("-" * 40)
for i, genre in enumerate(all_genres):
    count = user_preferences[1][i]
    if count > 0:  
        print(f"{genre}: {count}")

# Show top 5 preferred genres
print("\nUser 1's top preferred genres:")
user1_vector = user_preferences[1]
genre_counts = list(zip(all_genres, user1_vector))
sorted_genres = sorted(genre_counts, key=lambda x: x[1], reverse=True)

for genre, count in sorted_genres[:5]:
    if count > 0:
        print(f"{genre}: {count} movies")


Example - User 1 preferences:
----------------------------------------
Action: 76
Adventure: 74
Animation: 27
Children: 37
Comedy: 70
Crime: 39
Drama: 64
Fantasy: 41
Film-Noir: 1
Horror: 9
Musical: 20
Mystery: 13
Romance: 24
Sci-Fi: 32
Thriller: 43
War: 20
Western: 6

User 1's top preferred genres:
Action: 76 movies
Adventure: 74 movies
Comedy: 70 movies
Drama: 64 movies
Thriller: 43 movies


In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
import numpy as np

# 1. One‑hot encode all movies by genre
mlb = MultiLabelBinarizer(classes=all_genres)
genre_matrix = pd.DataFrame(
    mlb.fit_transform(movies['genre_list']),
    index=movies['movieId'],
    columns=all_genres
)

# 2. Normalize each user’s preference vector
user_profiles_norm = {}
for uid, vec in user_preferences.items():
    v = np.array(vec, dtype=float)
    n = norm(v)
    user_profiles_norm[uid] = (v / n) if n > 0 else v

# 3. For each user, score all unseen movies and pick Top‑K
K = 10
recommendations = {}

for uid, uvec in user_profiles_norm.items():
    # movies user has already rated (train+test)
    seen = set(ratings[ratings['userId'] == uid]['movieId'])
    
    # candidate pool = all movies minus seen
    candidates = genre_matrix.drop(index=seen, errors='ignore')
    
    # compute cosine similarity
    sims = cosine_similarity(uvec.reshape(1, -1), candidates.values)[0]
    
    # grab the Top‑K movieIds
    top_k_ids = candidates.index[np.argsort(sims)[-K:][::-1]]
    recommendations[uid] = list(top_k_ids)

# 4. Peeking at a user's top 10 films, the first being the highest, as a test.
print("User 1 recommendations:", recommendations.get(1, [])[:K])


User 1 recommendations: [117646, 55116, 6990, 5657, 4956, 4818, 42015, 164226, 2890, 82765]
